In [51]:
import pandas as pd
import numpy as np
import ast

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.impute import IterativeImputer

import warnings
warnings.filterwarnings("ignore")

In [52]:
def simpleImputation(data):
    for atr in range(len(data.columns)):
        data.iloc[:,atr] = data.iloc[:,atr].apply(lambda x: np.array(x))
        
    for atr in range(len(data.columns)):    
        mean_vector = np.sum(data.iloc[:,atr])/len(data) 
        for tup in range(len(data)):
            if(np.isnan(data.iloc[tup,atr]).all()): 
                data.iloc[tup,atr] = mean_vector
    for tup in data.index:
        for atr in data.columns:
                data.loc[tup,atr] = list(data.loc[tup,atr])
                
    return data

def knnImputation(data):
    for atr in range(len(data.columns)):
        data.iloc[:,atr] = data.iloc[:,atr].apply(lambda x: np.array(x))
        
    imputer_KNN = KNNImputer(n_neighbors=2)

    for atr in range(len(data.columns)):  
        
        len_vector = 0
        for tup in range(len(data)):
            if(np.isnan(data.iloc[tup,atr]).all() == False): len_vector = len(data.iloc[tup,atr])

        null_data = np.empty([len(data),len_vector])

        for tup in range(len(data)): null_data[tup] = data.iloc[tup,atr]

        imputed_data = (imputer_KNN.fit_transform(null_data))        
        for tup in range(len(data)):
                        
            if(np.isnan(data.iloc[tup,atr]).all()):
                data.iloc[tup,atr] = imputed_data[tup]    
                
    for tup in data.index:
        for atr in data.columns:
                data.loc[tup,atr] = list(data.loc[tup,atr])  
                
    return data  

def regressorImputation(data):
    
    for atr in range(len(data.columns)):
        data.iloc[:,atr] = data.iloc[:,atr].apply(lambda x: np.array(x))
        
    imputer_Reg = IterativeImputer(random_state=0, estimator=KNeighborsRegressor(n_neighbors=3))
    
    for atr in range(len(data.columns)):  

        if(pd.isnull(data.iloc[:,atr]).any()):
            len_vector = 0
            for tup in range(len(data)):
                if(np.isnan(data.iloc[tup,atr]).all() == False): len_vector = len(data.iloc[tup,atr])

            null_data = np.empty([len(data),len_vector])

            for tup in range(len(data)): null_data[tup] = data.iloc[tup,atr]

            imputed_data = imputer_Reg.fit_transform(null_data)

            for tup in range(len(data)): data.iloc[tup,atr] = imputed_data[tup]
 
    for tup in data.index:
        for atr in data.columns:
            data.loc[tup,atr] = list(data.loc[tup,atr])  
            
    return data

def decisionTreeImputation(data):
    for atr in range(len(data.columns)):
        data.iloc[:,atr] = data.iloc[:,atr].apply(lambda x: np.array(x))
    
    imputer_DT = IterativeImputer(random_state=0, estimator=DecisionTreeRegressor(max_features='sqrt', random_state=0))
    
    for atr in range(len(data.columns)):  

        if(pd.isnull(data.iloc[:,atr]).any()):
            len_vector = 0
            for tup in range(len(data)):
                if(np.isnan(data.iloc[tup,atr]).all() == False): len_vector = len(data.iloc[tup,atr])

            null_data = np.empty([len(data),len_vector])

            for tup in range(len(data)): null_data[tup] = data.iloc[tup,atr]

            imputed_data = imputer_DT.fit_transform(null_data)

            for tup in range(len(data)): data.iloc[tup,atr] = imputed_data[tup]
    
    for tup in data.index:
        for atr in data.columns:
            data.loc[tup,atr] = list(data.loc[tup,atr])  
            
    return data

In [65]:
### read csv data
# pathsData = ['../featured-data/ds-coil100/','../featured-data/ds-brats/', '../featured-data/ds-letters/']
pathsData = ['../featured-data/ds-letters/']

for ds in pathsData:
    print(ds.split('/')[2])
    for miss in [10, 50]:
        data = pd.read_csv(ds+'dataMissing{}.csv'.format(miss))
        ###### transform vector str to vectors of floats
        for tup in data.index:
            for atr in data.columns:
                # if(np.isnan(data.loc[tup,atr]).all() == False):
                if(type(data.loc[tup,atr]) == str and atr != 'class'):
                    # print(data.loc[tup,atr])
                    data.loc[tup,atr] = ast.literal_eval(data.loc[tup,atr])
        
        classCol = pd.read_csv('../featured-data/ds-letters/dataTesteComplete.csv')['class']
        data = data.iloc[:,1:]

        ## use an imputation method
        print('Mean Imputation')
        dataImp = simpleImputation(data.copy())
        dataImp.insert(0, 'class', classCol)
        dataImp.to_csv(ds+'dataTestMissing{}-SI.csv'.format(miss), index=False)
        

        print('Knn Imputation')
        dataImp = knnImputation(data.copy())
        dataImp.insert(0, 'class', classCol)
        dataImp.to_csv(ds+'dataTestMissing{}-KNN.csv'.format(miss), index=False)

        print('Regressor Imputation')
        dataImp = regressorImputation(data.copy())
        dataImp.insert(0, 'class', classCol)
        dataImp.to_csv(ds+'dataTestMissing{}-REG.csv'.format(miss), index=False)
        
        print('Decision Tree Imputation')
        dataImp = decisionTreeImputation(data.copy())
        dataImp.insert(0, 'class', classCol)
        dataImp.to_csv(ds+'dataTestMissing{}-DTI.csv'.format(miss), index=False)
        
        print()
print('OK')

ds-letters
Mean Imputation
Knn Imputation
Regressor Imputation


,class,A,a_l,B,b_l,C,c_l,D,d_l,E,...,u_l,V,v_l,W,w_l,X,x_l,Y,y_l,Z
0,ALGER,"[0.31830988618379125, 0.052117166807389516, 0....","[0.3183098861837899, 0.06578618426369907, 0.17...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9174678777869548, 147.90469517403372, 0.967...","[0.3183098861837918, 0.0046078845233988985, 0....","[0.3183098861837886, 0.012322154183169984, 0.0...","[0.3183098861837928, 0.0024863672617557397, 0....","[0.3183098861837901, 0.16599576439723235, 0.18...","[0.31830988618379025, 0.07134479573509388, 0.1...",...,"[0.31830988618379163, 0.0108855759899907, 0.04...","[0.9414077102022425, 40.4942344650865, 0.98694...","[0.9638530968589551, 40.235826393337604, 0.978...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9372472186928534, 60.71476617552851, 0.9813...","[0.31830988618379086, 0.02946101726982198, 0.3...","[0.9607506243710169, 77.6877802690583, 0.96057...","[0.9488043515368603, 49.842528827674556, 0.981...","[0.31830988618379097, 0.11382066226155096, 0.1...","[0.9484780264935184, 168.3100176169122, 0.9398..."
1,ANTQUAB,"[0.31830988618379025, 0.046087466133698816, 0....","[0.31830988618379025, 0.046087466133698816, 0....","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8840134938012625, 384.8254324151185, 0.9312...","[0.3183098861837918, 0.003851510668336075, 0.0...","[0.3183098861837918, 0.003851510668336075, 0.0...","[0.31830988618378964, 0.02977338341457361, 0.0...","[0.31830988618378964, 0.02977338341457361, 0.0...","[0.31830988618378936, 0.07314631102326677, 0.0...",...,"[0.31830988618378897, 0.0037834384067464313, 0...","[0.9080963023990538, 286.4949551569507, 0.9314...","[0.9080963023990538, 286.4949551569507, 0.9314...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8543263007503218, 275.3833279948751, 0.9605...","[0.3183098861837905, 0.30731657287711117, 0.20...","[0.9021114195396318, 389.6461402946828, 0.9121...","[0.9101833818681822, 257.2739830237027, 0.9381...","[0.31830988618378997, 0.1494648599773267, 0.09...","[0.9020884386374789, 403.7362668161435, 0.9137..."
2,ARIAL,"[0.3183098861837915, 0.1951847705827544, 0.068...","[0.3183098861837907, 0.11531268155325419, 0.03...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8998300710530283, 149.02146060217808, 0.973...","[0.31830988618378975, 0.001972725979151621, 0....","[0.3183098861837922, 0.044889426802059464, 0.0...","[0.31830988618378975, 0.0031457093562385114, 0...","[0.31830988618379175, 0.11297033524154966, 0.1...","[0.3183098861837921, 0.09852871144696643, 0.09...",...,"[0.3183098861837907, 0.004383928541482254, 0.0...","[0.9041094156508122, 173.49387411915438, 0.967...","[0.9387101589899783, 134.24647661755284, 0.960...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8975597813891727, 179.00944907110824, 0.968...","[0.31830988618379075, 0.5359195139244822, 0.55...","[0.9341908552918682, 153.74439461883406, 0.956...","[0.9106288715979, 165.7087203715567, 0.9667242...","[0.31830988618379064, 0.3847962420929538, 0.58...","[0.8936280238381826, 338.43405669442666, 0.943..."
3,ARIALN,"[0.31830988618379114, 0.12969833153140706, 0.0...","[0.31830988618379097, 0.1057324088098869, 0.24...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9141695243881917, 141.76541479820628, 0.970...","[0.31830988618378975, 0.004863245925651056, 0....","[0.3183098861837908, 0.0337021674581263, 0.037...","[0.3183098861837918, 0.0014002125578802262, 0....","[0.3183098861837893, 0.0016510894447885544, 0....","[0.31830988618378986, 0.13317167667099022, 0.0...",...,"[0.31830988618379064, 0.0037552951687802004, 0...","[0.9187910364321304, 92.65631005765532, 0.9790...","[0.943803893866555, 71.90390775144137, 0.97635...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9008420354921248, 104.47653747597693, 0.980...","[0.31830988618379075, 0.08631536307466098, 0.5...","[0.9330055566107716, 151.84561178731582, 0.957...","[0.9271411231030989, 117.0737508007687, 0.9707...","[0.3183098861837908, 0.2264717506

Decision Tree Imputation

Mean Imputation
Knn Imputation
Regressor Imputation


,class,A,a_l,B,b_l,C,c_l,D,d_l,E,...,u_l,V,v_l,W,w_l,X,x_l,Y,y_l,Z
0,ALGER,"[0.31830988618379025, 0.046087466133698816, 0....","[0.3183098861837908, 0.07354941497100749, 0.08...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8998556031845237, 168.23315716421095, 0.969...","[0.31830988618379114, 0.0031451342947923437, 0...","[0.31830988618379014, 0.07386903245447525, 0.0...","[0.31830988618379064, 0.0033467989403223037, 0...","[0.31830988618378964, 0.02977338341457361, 0.0...","[0.31830988618379, 0.10048757989807262, 0.1062...",...,"[0.31830988618378897, 0.0037834384067464313, 0...","[0.9080963023990538, 286.4949551569507, 0.9314...","[0.9373557224534101, 130.99209908178514, 0.961...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8543263007503218, 275.3833279948751, 0.9605...","[0.3183098861837905, 0.30731657287711117, 0.20...","[0.9021114195396318, 389.6461402946828, 0.9121...","[0.9291869901490227, 106.52370275464445, 0.972...","[0.3183098861837906, 0.06802814959085025, 0.27...","[0.9020884386374789, 403.7362668161435, 0.9137..."
1,ANTQUAB,"[0.3183098861837915, 0.1951847705827544, 0.068...","[0.3183098861837908, 0.07354941497100749, 0.08...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8998300710530283, 149.02146060217808, 0.973...","[0.31830988618379114, 0.0031451342947923437, 0...","[0.31830988618379014, 0.07386903245447525, 0.0...","[0.31830988618378975, 0.0031457093562385114, 0...","[0.3183098861837909, 0.09882616895252892, 0.08...","[0.31830988618379, 0.10048757989807262, 0.1062...",...,"[0.3183098861837916, 0.03986174701078605, 0.07...","[0.9041094156508122, 173.49387411915438, 0.967...","[0.9373557224534101, 130.99209908178514, 0.961...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8975597813891727, 179.00944907110824, 0.968...","[0.3183098861837906, 0.15995115884915842, 0.36...","[0.9341908552918682, 153.74439461883406, 0.956...","[0.9106288715979, 165.7087203715567, 0.9667242...","[0.31830988618379064, 0.3847962420929538, 0.58...","[0.9096707752692638, 231.22813901345293, 0.953..."
2,ARIAL,"[0.3183098861837912, 0.1244943036940308, 0.065...","[0.3183098861837908, 0.07354941497100749, 0.08...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9141695243881917, 141.76541479820628, 0.970...","[0.31830988618379114, 0.0031451342947923437, 0...","[0.31830988618378975, 0.004434178754468218, 0....","[0.3183098861837918, 0.0014002125578802262, 0....","[0.3183098861837893, 0.0016510894447885544, 0....","[0.3183098861837902, 0.08756965694326344, 0.33...",...,"[0.31830988618379064, 0.0037552951687802004, 0...","[0.9154700271549604, 72.76912502669228, 0.9847...","[0.943803893866555, 71.90390775144137, 0.97635...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8988223471518565, 121.03647501601536, 0.978...","[0.3183098861837906, 0.15995115884915842, 0.36...","[0.9330055566107716, 151.84561178731582, 0.957...","[0.9291869901490227, 106.52370275464445, 0.972...","[0.3183098861837906, 0.06802814959085025, 0.27...","[0.9096707752692638, 231.22813901345293, 0.953..."
3,ARIALN,"[0.3183098861837912, 0.1244943036940308, 0.065...","[0.3183098861837914, 0.08128828329069512, 0.28...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.9304952619585268, 111.6659593209481, 0.9711...","[0.31830988618379114, 0.0031451342947923437, 0...","[0.31830988618379014, 0.07386903245447525, 0.0...","[0.31830988618379064, 0.0008592261720497946, 0...","[0.31830988618379064, 0.0013796575897578884, 0...","[0.3183098861837909, 0.08936338948107687, 0.32...",...,"[0.3183098861837918, 0.004640459476490426, 0.0...","[0.9317726908447099, 69.67300608584239, 0.9808...","[0.9373557224534101, 130.99209908178514, 0.961...","[16515073.0, 16646145.0, 16711681.0, 16773121....","[0.8988223471518565, 121.03647501601536, 0.978...","[0.3183098861837906, 0.15995115884915842, 0.36...","[0.9310440179309271, 168.24798473200937, 0.955...","[0.9396557789087389, 89.41143497757847, 0.9724...","[0.31830988618379075, 0.270864915

Decision Tree Imputation

OK
